# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import json
import os
import numpy as np


# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
print("CSV Loaded Successfully")

# Display sample data
(city_data_df.head())

CSV Loaded Successfully


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display

# Configure the map plot

# Create a map using state centroid coordinates to set the center of the map
city_map = folium.Map(location = [0, 0], zoom_start=2)

#Plot each city on the map
for index, row in city_data_df.iterrows():

    #Create a circle marker for each city.
    folium.CircleMarker(
        location = [row["Lat"], row["Lng"]],
        radius = row["Humidity"] / 12,

        #Configure popup
        popup = (
            f"City: {row['City']}<br>"
            f"Country: {row['Country']}<br>"
            f"Humidity: {row['Humidity']}%"
        ),
        color = "yellow",
        fill = True,
        fill_color = "skyblue",
        fill_opacity = 0.8
    ).add_to(city_map)

# Display the map
city_map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[

    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 8) &
    (city_data_df["Cloudiness"] == 0)
]

ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275


In [14]:
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params =  {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
}
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
puerto del rosario - nearest hotel: Hotel Tamasite
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
pemba - nearest hotel: Hotel Raphael
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Hôtel Epona
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza Salparaíso
bandarbeyla - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot

hotel_vacation_map = folium.Map(location=[0, 0], zoom_start=2)

for index, row in hotel_df.iterrows():
    hotel = row["Hotel Name"]
    city = row["City"]
    country = row["Country"]
    lat = row["Lat"]
    lon = row["Lng"]

    popup_text = (
        f"<b>Hotel:</b> {hotel}<br>"
        f"<b>City:</b> {city}<br>"
        f"<b>Country:</b> {country}"
    )

#Add marker to the map

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color="red", icon="bed")
    ).add_to(hotel_vacation_map)


# Display the map
hotel_vacation_map
